In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [7]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [9]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [11]:
dataset=dataset.drop("User ID",axis=1)

In [13]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [15]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [17]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]

In [19]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [21]:
dependent=dataset[['Purchased']]

In [23]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [26]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [29]:
from sklearn.naive_bayes import BernoulliNB
from sklearn. model_selection import GridSearchCV
param_grid = {'alpha':[1], 
              'force_alpha':[True,False],
              'binarize':[0.0], 
              'fit_prior':[True,False], 
              'class_prior':[None]}
 
grid = GridSearchCV(BernoulliNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(x_train,y_train)  
print("Best parameters found: ", grid.best_params_)
y_pred = grid.predict(x_test)
print("Predictions: ", y_pred)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters found:  {'alpha': 1, 'binarize': 0.0, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}
Predictions:  [0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0 0
 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0 1]


C:\Users\user\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

                   

In [33]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [35]:
print(cm)

[[64 21]
 [ 9 40]]


In [37]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [39]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.88      0.75      0.81        85
           1       0.66      0.82      0.73        49

    accuracy                           0.78       134
   macro avg       0.77      0.78      0.77       134
weighted avg       0.80      0.78      0.78       134



In [41]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 1, 'binarize': 0.0, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}: 0.7798292770898099


In [43]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.8474189675870348

In [45]:
table=pd.DataFrame.from_dict(re)

In [47]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.038538,0.012861,0.015828,0.004964,1,0.0,None,True,True,"{'alpha': 1, 'binarize': 0.0, 'class_prior': N...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,3
1,0.021674,0.010269,0.017157,0.004421,1,0.0,None,True,False,"{'alpha': 1, 'binarize': 0.0, 'class_prior': N...",0.732497,0.719861,0.548024,0.749387,0.721794,0.694312,0.073891,3
2,0.006276,0.003913,0.014761,0.004427,1,0.0,None,False,True,"{'alpha': 1, 'binarize': 0.0, 'class_prior': N...",0.726166,0.686880,0.667268,0.759244,0.778179,0.723547,0.041899,1
3,0.008360,0.002264,0.015940,0.004785,1,0.0,None,False,False,"{'alpha': 1, 'binarize': 0.0, 'class_prior': N...",0.726166,0.686880,0.667268,0.759244,0.778179,0.723547,0.041899,1
